In [1]:
"""
snipper.io/s/-MSTCnlcnC3p7aUSnOm_
https://www.kaggle.com/c/clickbait-news-detection/

"""

'\nsnipper.io/s/-MSTCnlcnC3p7aUSnOm_\nhttps://www.kaggle.com/c/clickbait-news-detection/\n\n'

In [65]:
from copy import deepcopy

from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram, HashingTF, IDF, CountVectorizer, OneHotEncoder, StringIndexer, Word2Vec
from pyspark.sql.functions import col, udf, length, avg, lit, concat, size
from pyspark.sql.types import IntegerType, StringType
#from pyspark.mllib.classification import NaiveBayes#, NaiveBayesModel
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession

In [3]:
pyspark_stopwords = StopWordsRemover().getStopWords()#https://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.feature.StopWordsRemover
display(type(pyspark_stopwords), len(pyspark_stopwords), pyspark_stopwords)

list

181

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 '

In [4]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
try:
    sc.stop()
except:
    print("no sc to stop")
    
# try:
#     spark.stop()
# except Exception as e:
#     print("{}, {}".format(e,type(e)))
    

In [6]:
spark = SparkSession.builder.appName('ClickBait')\
    .getOrCreate()

In [7]:
sc, spark

(<SparkContext master=local[*] appName=PySparkShell>,
 <pyspark.sql.session.SparkSession at 0x7effec700fd0>)

In [8]:
train_path = "file:///home/hadoop/data/clickbait/train.csv"
#train_path = "/data/train.csv"
#train_path = "hdfs://localhost:9000/data/train.csv;"


In [9]:
# load main training dataframe

t_df = spark.read.csv(train_path, header=True, inferSchema=True).limit(5000)
t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [10]:
t_df.count()

5000

In [11]:
t_df=t_df.na.drop()

t_df.persist()

DataFrame[id: string, title: string, text: string, label: string]

In [12]:
t_df.count()

4965

In [39]:
t_df = t_df.withColumn("s", concat(col('title'), col('text')))



In [40]:
t_df.select('s').limit(20).show()

+--------------------+
|                   s|
+--------------------+
|China and Economi...|
|Trade to Be a Big...|
|The Top Beaches I...|
|Sheriff’s Report ...|
|Surgeon claiming ...|
|This Is How Diffe...|
|Samantha Bee and ...|
|Krauthammer: Syri...|
|Rust Belt voters ...|
|As Illegal Outpos...|
|$94.4m $198m $112...|
|Trump’s Mar-a-Lag...|
|STOCKS SLIP TO EN...|
|Jared Kushner in ...|
|He Cheated. Now H...|
|The fabulous life...|
|Destinations Insi...|
|President Donald ...|
|4 Things That You...|
|Freddie Flintoff ...|
+--------------------+



In [64]:
selection = ['s']

t_df \
    .select(*(length(col(c)).alias(c) for c in selection)) \
    .agg(*(avg(col(c)).alias(c) for c in selection)).show()



+-----------------+
|                s|
+-----------------+
|2500.170392749245|
+-----------------+

Help on function length in module pyspark.sql.functions:

length(col)
    Computes the character length of string data or number of bytes of binary data.
    The length of character data includes the trailing spaces. The length of binary data
    includes binary zeros.
    
    >>> spark.createDataFrame([('ABC ',)], ['a']).select(length('a').alias('length')).collect()
    [Row(length=4)]
    
    .. versionadded:: 1.5



In [71]:
from collections.abc import Iterable

def get_avg_len_cols(df, selection):
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(length(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

def get_avg_size_cols(df, selection):
    assert isinstance(selection, Iterable)
    return(
        df \
        .select(*(size(col(c)).alias(c) for c in selection)) \
        .agg(*(avg(col(c)).alias(c) for c in selection)).show()
              )

In [48]:
t_df = Tokenizer(inputCol = "s", outputCol = "ss").transform(t_df)

"""
some punctuation like ',' is still showing after Tokenizer().transform.
"""

IllegalArgumentException: 'Output column ss already exists.'

In [72]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- s: string (nullable = true)
 |-- ss: array (nullable = true)
 |    |-- element: string (containsNull = true)



+------------------+
|                ss|
+------------------+
|415.75649546827793|
+------------------+



In [74]:


get_avg_size_cols(t_df, ['ss'])

+------------------+
|                ss|
+------------------+
|415.75649546827793|
+------------------+



In [75]:
t_df = StopWordsRemover(inputCol = "ss", outputCol="sss").transform(t_df)

In [76]:
get_avg_size_cols(t_df, ['sss'])

+-----------------+
|              sss|
+-----------------+
|251.9915407854985|
+-----------------+



In [77]:
t_df.select('sss').limit(5).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [79]:
t_df.sample(False, 0.1, seed=0).limit(1)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:42577)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:42577)

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[*] appName=PySparkShell>

In [42]:
# Preprocessing Strategy
"""
plan is to turn this into a Pipeline([Tokenizer,StopWordsRemover,Ngram1,Ngram2...])
"""
raise AssertionError('stop')

#count_tokens = udf(lambda words : len(words) , IntegerType())
text_cols= ["text","title"]
for t_c in text_cols:
    t_df = Tokenizer(inputCol = t_c, outputCol = "token_"+t_c).transform(t_df)
    t_df = StopWordsRemover(inputCol = "token_"+t_c, outputCol = t_c+"_f_token").transform(t_df)
    
    for i in range(1,4):
        t_df= NGram(n=i, inputCol=t_c+"_f_token", outputCol=t_c+"_ngram_"+str(i)).transform(t_df)
        
        #CountVectorizer for each ngram col
#         c_v = CountVectorizer(inputCol= t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"cvfeat",
#                               minDF=2.0, vocabSize=2**14)
        
        #indexer = StringIndexer(inputCol=t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"ix_feat")
        #display(t_c+"_ngram_"+str(i), t_c+"_ngram_"+str(i)+"cvfeat")
        
        #m_ = c_v.fit(t_df) # c_v.fit is failing on 
        #t_df = m_.tranform(t_df)
        #m_ = c_v.fit(t_df)
        #t_df = m_.transform(t_df)
    
    t_df = t_df.drop("token_"+t_c)



t_df.persist()

AssertionError: stop

In [14]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- text_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [15]:
# # Preprocessing Strategy
# text_cols= ["text","title"]

# for t_c in text_cols:
#     t_df = Tokenizer(inputCol = t_c, outputCol = "token_"+t_c).transform(t_df)
#     t_df = StopWordsRemover(inputCol = "token_"+t_c, outputCol = t_c+"_f_token").transform(t_df)
    
#     for i in range(1,4):
#         t_df= NGram(n=i, inputCol=t_c+"_f_token", outputCol=t_c+"_ngram_"+str(i)).transform(t_df)
        
#         #CountVectorizer for each ngram col
#         c_v = CountVectorizer(inputCol= t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"cvfeat",
#                               minDF=2.0, vocabSize=2**14)
        
#         #indexer = StringIndexer(inputCol=t_c+"_ngram_"+str(i), outputCol=t_c+"_ngram_"+str(i)+"ix_feat")
#         #display(t_c+"_ngram_"+str(i), t_c+"_ngram_"+str(i)+"cvfeat")
        
#         #m_ = c_v.fit(t_df) # c_v.fit is failing
#         #t_df = m_.tranform(t_df)
        

#     t_df = t_df.drop("token_"+t_c)


# #t_df.show()


# # def build_ngrams_wocs(inputCol=["text","target"], n=3):
# #     tokenizer = [Tokenizer(inputCol="text", outputCol="words")]
# #     ngrams = [
# #         NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
# #         for i in range(1, n + 1)
# #     ]

# #     cv = [
# #         CountVectorizer(vocabSize=5460,inputCol="{0}_grams".format(i),
# #             outputCol="{0}_tf".format(i))
# #         for i in range(1, n + 1)
# #     ]
# #     idf = [IDF(inputCol="{0}_tf".format(i), outputCol="{0}_tfidf".format(i), minDocFreq=5) for i in range(1, n + 1)]

# #     assembler = [VectorAssembler(
# #         inputCols=["{0}_tfidf".format(i) for i in range(1, n + 1)],
# #         outputCol="features"
# #     )]
# #     label_stringIdx = [StringIndexer(inputCol = "target", outputCol = "label")]
# #     lr = [LogisticRegression(maxIter=100)]
# #     return Pipeline(stages=tokenizer + ngrams + cv + idf+ assembler + label_stringIdx+lr)



In [16]:
#t_df.select("title_ngram_3").show(5, truncate=False)

t_df.select("text_ngram_1").limit(1).collect()[0]['text_ngram_1']

{type(z) for z in t_df.select("text_ngram_1").limit(1).collect()[0]['text_ngram_1']}
#, t_df.columns

{str}

In [17]:
t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- text_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [18]:
c_v = CountVectorizer(inputCol="text_ngram_2", outputCol="999998",
                     minDF=2.0, vocabSize=2**14)

# _p = {'inputCol':"title_ngram_1", 'outputCol':"title_ngram_1_cv",
#       'minDF':2.0, 'vocabSize':2**14}

#m_ = c_v.fit(t_df, params={'inputCol': 'text_ngram_1'})
mdl = c_v.fit(t_df)

t_df=mdl.transform(t_df)
t_df.persist()

DataFrame[id: string, title: string, text: string, label: string, text_f_token: array<string>, text_ngram_1: array<string>, text_ngram_2: array<string>, text_ngram_3: array<string>, title_f_token: array<string>, title_ngram_1: array<string>, title_ngram_2: array<string>, title_ngram_3: array<string>, 999998: vector]

In [19]:
# df = spark.createDataFrame(
#     [(0, ["a", "b a", "c"]), (1, ["a", "b", "b", "c", "a"])],
#     ["label", "raw"])
# cv = CountVectorizer(inputCol="raw", outputCol="vectors")
# model = cv.fit(df)
# model.transform(df).show(truncate=False)

In [20]:
t_df.printSchema(), t_df.columns

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- text_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 999998: vector (nullable = true)



(None,
 ['id',
  'title',
  'text',
  'label',
  'text_f_token',
  'text_ngram_1',
  'text_ngram_2',
  'text_ngram_3',
  'title_f_token',
  'title_ngram_1',
  'title_ngram_2',
  'title_ngram_3',
  '999998'])

In [21]:
#text_tokenizer = Tokenizer(inputCol = 'text', outputCol = 'text_words')
#ttl_tokenizer = Tokenizer(inputCol = 'title', outputCol = 'ttl_words')
#regex_tokenizer = RegexTokenizer(inputCol= 'sentence', outputCol= 'words', pattern='\\W')
#t_df = tokenizer.transform(t_df)
# t_df = text_tokenizer.transform(ttl_tokenizer.transform(t_df))
help(CountVectorizer)

Help on class CountVectorizer in module pyspark.ml.feature:

class CountVectorizer(pyspark.ml.wrapper.JavaEstimator, _CountVectorizerParams, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 |  Extracts a vocabulary from document collections and generates a :py:attr:`CountVectorizerModel`.
 |  
 |  >>> df = spark.createDataFrame(
 |  ...    [(0, ["a", "b", "c"]), (1, ["a", "b", "b", "c", "a"])],
 |  ...    ["label", "raw"])
 |  >>> cv = CountVectorizer(inputCol="raw", outputCol="vectors")
 |  >>> model = cv.fit(df)
 |  >>> model.transform(df).show(truncate=False)
 |  +-----+---------------+-------------------------+
 |  |label|raw            |vectors                  |
 |  +-----+---------------+-------------------------+
 |  |0    |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
 |  |1    |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
 |  +-----+---------------+-------------------------+
 |  ...
 |  >>> sorted(model.vocabulary) == ['a', 'b', 'c']
 |  True
 |  >>> countVectorize

In [22]:
## Feature building

fcols = [
#  'text_f_token',
  'text_ngram_1',
  'text_ngram_2',
  'text_ngram_3',
#  'title_f_token',
  'title_ngram_1',
  'title_ngram_2',
  'title_ngram_3']

indexer = StringIndexer(inputCol="label", outputCol="label_index")
t_df = indexer.fit(t_df).transform(t_df)

cv = CountVectorizer(inputCol="text_ngram_2", outputCol="features")
t_df=cv.fit(t_df).transform(t_df)

t_df.persist()



# ohe = OneHotEncoderEstimator(
#     inputCols= fcols,
#     outputCols=["feat_"+s for s in fcols])

#f_df=ohe.fit(t_df).transform(t_df)
    
#f_df=cv.fit(t_df).transform(t_df)
#OneHotEncoderEstimator
#hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

DataFrame[id: string, title: string, text: string, label: string, text_f_token: array<string>, text_ngram_1: array<string>, text_ngram_2: array<string>, text_ngram_3: array<string>, title_f_token: array<string>, title_ngram_1: array<string>, title_ngram_2: array<string>, title_ngram_3: array<string>, 999998: vector, label_index: double, features: vector]

In [23]:
# idf= IDF(inputCol='rawFeatures', outputCol='features')
# idf_model = idf.fit(featurized_df)
# rescaled_df = idf_model.transform(featurized_df)

# rescaled_df.show(truncate=False)

t_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- text_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- text_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_f_token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title_ngram_1: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_2: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- title_ngram_3: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 999998: vector (nullable = true)
 |-- label_index: double (nullable = false)
 |-- features: vector (nullable

In [24]:
help(NaiveBayes)

Help on class NaiveBayes in module pyspark.ml.classification:

class NaiveBayes(pyspark.ml.wrapper.JavaEstimator, pyspark.ml.param.shared.HasFeaturesCol, pyspark.ml.param.shared.HasLabelCol, pyspark.ml.param.shared.HasPredictionCol, pyspark.ml.param.shared.HasProbabilityCol, pyspark.ml.param.shared.HasRawPredictionCol, pyspark.ml.param.shared.HasThresholds, pyspark.ml.param.shared.HasWeightCol, pyspark.ml.util.JavaMLWritable, pyspark.ml.util.JavaMLReadable)
 |  Naive Bayes Classifiers.
 |  It supports both Multinomial and Bernoulli NB. `Multinomial NB
 |  <http://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html>`_
 |  can handle finitely supported discrete data. For example, by converting documents into
 |  TF-IDF vectors, it can be used for document classification. By making every vector a
 |  binary (0/1) data, it can also be used as `Bernoulli NB
 |  <http://nlp.stanford.edu/IR-book/html/htmledition/the-bernoulli-model-1.html>`_.
 |  The input feature

In [25]:
train, test = t_df.randomSplit([.8,.2], seed = 999)

In [26]:
NB = NaiveBayes(modelType= "multinomial", labelCol="label_index", featuresCol ="999998")

In [27]:
nb_model= NB.fit(train)
nb_predictions= nb_model.transform(test)



"""
Py4JJavaError: An error occurred while calling o459.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 23, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
"""

Py4JJavaError: An error occurred while calling o459.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 23, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1786)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1189)
	at java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:348)
	at org.apache

In [ ]:

evaluator = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "prediction", 
                                              metricName="accuracy")

nb_accuracy = evaluator.evaluate(nb_predictions)

print("Accuracy : ", nb_accuracy)

In [ ]:
"""
https://spark.apache.org/docs/2.2.0/ml-pipeline.html

Plan:
    - use Pipeline([transformer_n, estimator_n...])
    - Improve preprocessing (punctuation, special characters, stemming)
    - Include transformers and estimators for each n_gram length

"""